## Import libraries

In [123]:
import io
import itertools

import numpy as np
import sklearn.metrics

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

import matplotlib.pyplot as plt

## Load and preprocess data

In [124]:
train_data = np.load('dataset/Trousers & Jeans - All - Train.npz')
validation_data = np.load('dataset/Trousers & Jeans - All - Validation.npz')
test_data = np.load('dataset/Trousers & Jeans - All - Test.npz')

In [125]:
list(train_data.keys())

['images', 'labels', 'genders']

In [126]:
images_train, labels_train = train_data['images'], train_data['labels']
images_val, labels_val = validation_data['images'], validation_data['labels']
images_test, labels_test = test_data['images'], test_data['labels']

In [127]:
np.unique(labels_train)

array([0, 1, 2, 3])

In [128]:
images_train.shape

(4004, 120, 90, 3)

In [129]:
labels_train.shape

(4004,)

In [130]:
images_val.shape

(500, 120, 90, 3)

In [131]:
labels_val.shape

(500,)

In [132]:
images_test.shape

(500, 120, 90, 3)

In [133]:
labels_test.shape

(500,)

In [134]:
# standardize 
images_train = images_train/255.0
images_val = images_val/255.0
images_test = images_test/255.0

## Create the model!

### Define hyparameters

In [135]:
EPOCHS = 15
BATCH_SIZE = 64

In [136]:
HP_KERNEL_SIZE_1 = hp.HParam('kernel size 1', hp.Discrete([3,5,7]))
HP_KERNEL_SIZE_2 = hp.HParam('kernel size 2', hp.Discrete([3,5]))
HP_KERNEL_NUM = hp.HParam('kernel num', hp.Discrete([32,64,96,128]))
HP_DENSE_SIZE = hp.HParam('dense size', hp.Discrete([256,512,1024]))

In [137]:
METRIC = 'accuracy'

with tf.summary.create_file_writer(r'Logs/Model 3 (Trousers&Jeans_All)/hparam_tuning/').as_default():
    hp.hparams_config(
        hparams = [HP_KERNEL_SIZE_1, HP_KERNEL_SIZE_2, HP_KERNEL_NUM, HP_DENSE_SIZE],
        metrics = [hp.Metric(METRIC, display_name='accuracy')]
    )

### Model architecture

In [138]:
def train_test_model(hparams, session_num):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(hparams[HP_KERNEL_NUM], hparams[HP_KERNEL_SIZE_1], activation = 'relu', input_shape = (120,90,3)),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(hparams[HP_KERNEL_NUM], hparams[HP_KERNEL_SIZE_2], activation = 'relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(hparams[HP_DENSE_SIZE], activation = 'relu'),
        tf.keras.layers.Dense(4, activation='softmax')
    ])
    
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
    model.compile(optimizer = 'adam', loss = loss_fn, metrics = ['accuracy'])
    
    log_dir = "Logs\\Model 3 (Trousers&Jeans_All)\\fit\\" + f"run-{session_num}"
    
    # functions for creating confusion matrix
    def plot_confusion_matrix(cm, class_names):
        """
        Returns a matplotlib figure containing the plotted confusion matrix.

        Args:
        cm (array, shape = [n, n]): a confusion matrix of integer classes
        class_names (array, shape = [n]): String names of the integer classes
        """
        figure = plt.figure(figsize=(12, 12))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title("Confusion matrix")
        plt.colorbar()
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names, rotation=45)
        plt.yticks(tick_marks, class_names)

        # Normalize the confusion matrix.
        cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

        # Use white text if squares are dark; otherwise black.
        threshold = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            color = "white" if cm[i, j] > threshold else "black"
            plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')

        return figure

    def plot_to_image(figure):
        """Converts the matplotlib plot specified by 'figure' to a PNG image and
        returns it. The supplied figure is closed and inaccessible after this call."""

        # Save the plot to a PNG in memory.
        buf = io.BytesIO()
        plt.savefig(buf, format='png')

        # Closing the figure prevents it from being displayed directly inside the notebook.
        plt.close(figure)

        buf.seek(0)

        # Convert PNG buffer to TF image
        image = tf.image.decode_png(buf.getvalue(), channels=4)

        # Add the batch dimension
        image = tf.expand_dims(image, 0)

        return image

    # Define a file writer variable for logging purposes
    file_writer_cm = tf.summary.create_file_writer(log_dir + '/cm')

    def log_confusion_matrix(epoch, logs):
        # Use the model to predict the values from the validation dataset.
        test_pred_raw = model.predict(images_val)
        test_pred = np.argmax(test_pred_raw, axis=1)

        # Calculate the confusion matrix.
        cm = sklearn.metrics.confusion_matrix(labels_val, test_pred)

        # Log the confusion matrix as an image summary.
        figure = plot_confusion_matrix(cm, class_names=['Trousers Male','Jeans Male','Trousers Female','Jeans Female'])
        cm_image = plot_to_image(figure)

        # Log the confusion matrix as an image summary.
        with file_writer_cm.as_default():
            tf.summary.image("Confusion Matrix", cm_image, step=epoch)
            
    # callbacks
    early_stopping = tf.keras.callbacks.EarlyStopping(patience = 2, restore_best_weights = True)
    cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end = log_confusion_matrix)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq = 1, profile_batch = 0)
    
    model.fit(images_train,
              labels_train,
              epochs = EPOCHS,
              batch_size = BATCH_SIZE,
              callbacks = [tensorboard_callback, cm_callback, early_stopping],
              validation_data = (images_val, labels_val),
              verbose = 1)
    
    _, accuracy = model.evaluate(images_val, labels_val)
    
    model.save(r'saved_models\Model 3 (Trousers&Jeans_All)\Run-{}'.format(session_num))
    
    return accuracy

In [139]:
def run(log_dir, hparams, session_num):
    with tf.summary.create_file_writer(log_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams, session_num)
        tf.summary.scalar(METRIC,accuracy,step=1)

In [140]:
session_num = 1
for kernel_size_1 in HP_KERNEL_SIZE_1.domain.values:
    for kernel_num in HP_KERNEL_NUM.domain.values:
        for kernel_size_2 in HP_KERNEL_SIZE_2.domain.values:
            for dense_size in HP_DENSE_SIZE.domain.values:
                hparams = {
                    HP_KERNEL_SIZE_1 : kernel_size_1,
                    HP_KERNEL_SIZE_2 : kernel_size_2,
                    HP_KERNEL_NUM : kernel_num,
                    HP_DENSE_SIZE : dense_size
                }
                run_name = f'run-{session_num}'
                print('--- Starting Trial:',run_name)
                print({h.name: hparams[h] for h in hparams})
                run("Logs/Model 3 (Trousers&Jeans_All)/hparam_tuning/" + run_name, hparams, session_num)

                session_num += 1

--- Starting Trial: run-1
{'kernel size 1': 3, 'kernel size 2': 3, 'kernel num': 32, 'dense size': 256}
Epoch 1/15
63/63 [==============================] - 21s 327ms/step - loss: 1.1822 - accuracy: 0.5040 - val_loss: 1.0516 - val_accuracy: 0.5660
Epoch 2/15
63/63 [==============================] - 21s 333ms/step - loss: 0.8284 - accuracy: 0.6673 - val_loss: 0.8749 - val_accuracy: 0.6540
Epoch 3/15
63/63 [==============================] - 22s 342ms/step - loss: 0.7099 - accuracy: 0.7178 - val_loss: 0.9516 - val_accuracy: 0.6180
Epoch 4/15
63/63 [==============================] - 20s 323ms/step - loss: 0.6503 - accuracy: 0.7390 - val_loss: 0.8365 - val_accuracy: 0.6860
Epoch 5/15
63/63 [==============================] - 20s 314ms/step - loss: 0.5930 - accuracy: 0.7617 - val_loss: 0.7883 - val_accuracy: 0.6760
Epoch 6/15
63/63 [==============================] - 20s 314ms/step - loss: 0.5261 - accuracy: 0.7937 - val_loss: 0.8290 - val_accuracy: 0.6960
Epoch 7/15
16/16 [====================

Epoch 9/15
63/63 [==============================] - 72s 1s/step - loss: 0.5477 - accuracy: 0.7847 - val_loss: 0.7293 - val_accuracy: 0.7300
Epoch 10/15
63/63 [==============================] - 86s 1s/step - loss: 0.4981 - accuracy: 0.8034 - val_loss: 0.7088 - val_accuracy: 0.7260
Epoch 11/15
63/63 [==============================] - 87s 1s/step - loss: 0.5260 - accuracy: 0.7982 - val_loss: 0.8002 - val_accuracy: 0.6920
Epoch 12/15
16/16 [==============================] - 2s 119ms/step - loss: 0.7088 - accuracy: 0.7260
INFO:tensorflow:Assets written to: saved_models\Model 3 (Trousers&Jeans_All)\Run-11\assets
--- Starting Trial: run-12
{'kernel size 1': 3, 'kernel size 2': 5, 'kernel num': 64, 'dense size': 1024}
Epoch 1/15
63/63 [==============================] - 76s 1s/step - loss: 1.5700 - accuracy: 0.4980 - val_loss: 1.0421 - val_accuracy: 0.5880
Epoch 2/15
63/63 [==============================] - 77s 1s/step - loss: 0.8514 - accuracy: 0.6479 - val_loss: 0.9157 - val_accuracy: 0.6120


63/63 [==============================] - 157s 2s/step - loss: 0.6868 - accuracy: 0.7188 - val_loss: 0.8635 - val_accuracy: 0.6520
Epoch 6/15
16/16 [==============================] - 5s 301ms/step - loss: 0.7800 - accuracy: 0.6780
INFO:tensorflow:Assets written to: saved_models\Model 3 (Trousers&Jeans_All)\Run-23\assets
--- Starting Trial: run-24
{'kernel size 1': 3, 'kernel size 2': 5, 'kernel num': 128, 'dense size': 1024}
Epoch 1/15
63/63 [==============================] - 169s 3s/step - loss: 2.3118 - accuracy: 0.4233 - val_loss: 1.1701 - val_accuracy: 0.4800
Epoch 2/15
63/63 [==============================] - 168s 3s/step - loss: 0.9717 - accuracy: 0.5947 - val_loss: 0.9518 - val_accuracy: 0.6020
Epoch 3/15
63/63 [==============================] - 168s 3s/step - loss: 0.8321 - accuracy: 0.6573 - val_loss: 0.8986 - val_accuracy: 0.6440
Epoch 4/15
63/63 [==============================] - 168s 3s/step - loss: 0.7668 - accuracy: 0.6886 - val_loss: 0.8831 - val_accuracy: 0.6420
Epoch 5/

63/63 [==============================] - 59s 943ms/step - loss: 0.8292 - accuracy: 0.6666 - val_loss: 0.8922 - val_accuracy: 0.6380
Epoch 4/15
63/63 [==============================] - 60s 947ms/step - loss: 0.7582 - accuracy: 0.6863 - val_loss: 0.9147 - val_accuracy: 0.6100
Epoch 5/15
63/63 [==============================] - 60s 950ms/step - loss: 0.7071 - accuracy: 0.7038 - val_loss: 0.8767 - val_accuracy: 0.6560
Epoch 6/15
63/63 [==============================] - 60s 957ms/step - loss: 0.6522 - accuracy: 0.7395 - val_loss: 0.8494 - val_accuracy: 0.6560
Epoch 7/15
63/63 [==============================] - 59s 944ms/step - loss: 0.6149 - accuracy: 0.7473 - val_loss: 0.7830 - val_accuracy: 0.6980
Epoch 8/15
63/63 [==============================] - 60s 946ms/step - loss: 0.5756 - accuracy: 0.7625 - val_loss: 0.8590 - val_accuracy: 0.6960
Epoch 9/15
16/16 [==============================] - 2s 112ms/step - loss: 0.7830 - accuracy: 0.6980
INFO:tensorflow:Assets written to: saved_models\Model

63/63 [==============================] - 80s 1s/step - loss: 0.7966 - accuracy: 0.6708 - val_loss: 0.9515 - val_accuracy: 0.5800
Epoch 4/15
63/63 [==============================] - 80s 1s/step - loss: 0.8137 - accuracy: 0.6643 - val_loss: 0.8236 - val_accuracy: 0.6360
Epoch 5/15
63/63 [==============================] - 80s 1s/step - loss: 0.7044 - accuracy: 0.7093 - val_loss: 0.8195 - val_accuracy: 0.6560
Epoch 6/15
63/63 [==============================] - 82s 1s/step - loss: 0.6745 - accuracy: 0.7283 - val_loss: 0.7830 - val_accuracy: 0.6780
Epoch 7/15
63/63 [==============================] - 93s 1s/step - loss: 0.5918 - accuracy: 0.7620 - val_loss: 0.7696 - val_accuracy: 0.7020
Epoch 8/15
63/63 [==============================] - 100s 2s/step - loss: 0.5239 - accuracy: 0.7987 - val_loss: 0.8088 - val_accuracy: 0.6880
Epoch 9/15
63/63 [==============================] - 100s 2s/step - loss: 0.5238 - accuracy: 0.7902 - val_loss: 0.7345 - val_accuracy: 0.7200
Epoch 10/15
63/63 [==========

63/63 [==============================] - 121s 2s/step - loss: 0.7514 - accuracy: 0.6923 - val_loss: 1.0105 - val_accuracy: 0.5980
Epoch 5/15
63/63 [==============================] - 123s 2s/step - loss: 0.7111 - accuracy: 0.7098 - val_loss: 0.8395 - val_accuracy: 0.6440
Epoch 6/15
63/63 [==============================] - 132s 2s/step - loss: 0.6271 - accuracy: 0.7505 - val_loss: 0.7664 - val_accuracy: 0.7020
Epoch 7/15
63/63 [==============================] - 124s 2s/step - loss: 0.6261 - accuracy: 0.7428 - val_loss: 0.8096 - val_accuracy: 0.7020
Epoch 8/15
63/63 [==============================] - 121s 2s/step - loss: 0.5618 - accuracy: 0.7697 - val_loss: 0.7651 - val_accuracy: 0.7080
Epoch 9/15
63/63 [==============================] - 120s 2s/step - loss: 0.5272 - accuracy: 0.7835 - val_loss: 0.7450 - val_accuracy: 0.7180
Epoch 10/15
63/63 [==============================] - 125s 2s/step - loss: 0.4806 - accuracy: 0.8029 - val_loss: 0.7856 - val_accuracy: 0.7260
Epoch 11/15
16/16 [====

--- Starting Trial: run-50
{'kernel size 1': 7, 'kernel size 2': 3, 'kernel num': 32, 'dense size': 512}
Epoch 1/15
63/63 [==============================] - 30s 474ms/step - loss: 1.2649 - accuracy: 0.4700 - val_loss: 1.1585 - val_accuracy: 0.5320
Epoch 2/15
63/63 [==============================] - 30s 472ms/step - loss: 0.9352 - accuracy: 0.6126 - val_loss: 0.9314 - val_accuracy: 0.6140
Epoch 3/15
63/63 [==============================] - 33s 523ms/step - loss: 0.7983 - accuracy: 0.6811 - val_loss: 0.9042 - val_accuracy: 0.6180
Epoch 4/15
63/63 [==============================] - 31s 485ms/step - loss: 0.7544 - accuracy: 0.6941 - val_loss: 0.8758 - val_accuracy: 0.6440
Epoch 5/15
63/63 [==============================] - 29s 463ms/step - loss: 0.6860 - accuracy: 0.7205 - val_loss: 0.8577 - val_accuracy: 0.6700
Epoch 6/15
63/63 [==============================] - 29s 462ms/step - loss: 0.6204 - accuracy: 0.7560 - val_loss: 0.9422 - val_accuracy: 0.6300
Epoch 7/15
63/63 [===================

63/63 [==============================] - 65s 1s/step - loss: 0.6291 - accuracy: 0.7500 - val_loss: 0.7404 - val_accuracy: 0.7140
Epoch 8/15
63/63 [==============================] - 65s 1s/step - loss: 0.5935 - accuracy: 0.7662 - val_loss: 0.8072 - val_accuracy: 0.6960
Epoch 9/15
16/16 [==============================] - 2s 125ms/step - loss: 0.7404 - accuracy: 0.7140
INFO:tensorflow:Assets written to: saved_models\Model 3 (Trousers&Jeans_All)\Run-59\assets
--- Starting Trial: run-60
{'kernel size 1': 7, 'kernel size 2': 5, 'kernel num': 64, 'dense size': 1024}
Epoch 1/15
63/63 [==============================] - 72s 1s/step - loss: 1.5428 - accuracy: 0.4281 - val_loss: 1.1475 - val_accuracy: 0.4720
Epoch 2/15
63/63 [==============================] - 72s 1s/step - loss: 1.0312 - accuracy: 0.5672 - val_loss: 0.9984 - val_accuracy: 0.5680
Epoch 3/15
63/63 [==============================] - 72s 1s/step - loss: 0.8783 - accuracy: 0.6366 - val_loss: 0.9239 - val_accuracy: 0.6140
Epoch 4/15
63/

--- Starting Trial: run-70
{'kernel size 1': 7, 'kernel size 2': 5, 'kernel num': 128, 'dense size': 256}
Epoch 1/15
63/63 [==============================] - 154s 2s/step - loss: 1.4037 - accuracy: 0.4573 - val_loss: 0.9960 - val_accuracy: 0.6080
Epoch 2/15
63/63 [==============================] - 154s 2s/step - loss: 0.8940 - accuracy: 0.6319 - val_loss: 0.9506 - val_accuracy: 0.6280
Epoch 3/15
63/63 [==============================] - 154s 2s/step - loss: 0.8534 - accuracy: 0.6446 - val_loss: 0.9454 - val_accuracy: 0.6340
Epoch 4/15
63/63 [==============================] - 153s 2s/step - loss: 0.7738 - accuracy: 0.6828 - val_loss: 0.8369 - val_accuracy: 0.6880
Epoch 5/15
63/63 [==============================] - 154s 2s/step - loss: 0.6997 - accuracy: 0.7123 - val_loss: 0.8396 - val_accuracy: 0.6820
Epoch 6/15
63/63 [==============================] - 153s 2s/step - loss: 0.6517 - accuracy: 0.7393 - val_loss: 0.7975 - val_accuracy: 0.6680
Epoch 7/15
63/63 [==============================